In [1]:
# membaca file, kemudian teksnya disimpan ke dalam list

import numpy
import pandas
import os

row_list = []
for subdir in ['bagus','jelek']:
    for folder, subfolders, filenames in os.walk('review_film/'+subdir):
        for file in filenames:  
            d = {'class':subdir}  
            with open('review_film/'+subdir+'/'+file) as f:
                if f.read():      
                    f.seek(0)
                    d['text'] = f.read()  
            row_list.append(d)
        break

In [2]:
# list teks tersebut disimpan ke dalam dataframe

dataframe = pandas.DataFrame(row_list)

In [3]:
# melihat isi dataframe

dataframe.head()

,class,text
0,bagus,films adapted from comic books have had plenty...
1,bagus,every now and then a movie comes along from a ...
2,bagus,you've got mail works alot better than it dese...
3,bagus,""" jaws "" is a rare film that grabs your atten..."
4,bagus,moviemaking is a lot like being the general ma...


In [4]:
# ekstraksi fitur pada teks

from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(dataframe['text'])

In [5]:
# menerapkan Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = dataframe['class']
classifier.fit(counts, targets)

MultinomialNB()

In [6]:
# membuat prediksi untuk 2 contoh (jelek dan bagus)
examples = ["the law of crowd pleasing romantic movies states that the two leads must end up togetherby film's end .if you're not familiar with this law , then maybe you've seen the trailer for this film which shows that the two leads are together by film's end . now if you're a regular reader of mine , you've heard me say this countless times : you know how drive me crazy is going to end , but is the journey to get to that ending worth it ? no , it definitely is not . melissa joan hart ( from abc's  sabrina , the teenage witch  ) likes a hunky stud on the basketball team . adrien grenier is her grungy neighbor who's just broken up with his activist girlfriend . apparently he wants to make his ex-girlfriend jealous enough to take him back , and she wants someone to take her to the big year end dance ."
            , "this three hour movie opens up with a view of singer/guitar player/musician/composer frank zappa rehearsing with his fellow band members . all the rest displays a compilation of footage , mostly from the concert at the palladium in new york city , halloween 1979 . other footage shows backstage foolishness , and amazing clay animation by bruce bickford . the performance of \" titties and beer \" played in this movie is very entertaining , with drummer terry bozzio supplying the voice of the devil . frank's guitar solos outdo any van halen or hendrix i've ever heard . bruce bickford's outlandish clay animation is that beyond belief with zooms , morphings , etc . and actually , it doesn't even look like clay , it looks like meat ."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
print(predictions)


##Untuk memprediksi review pada soal nomor 2, yaitu dengan kode di bawah ini:

#tes = ["the law of crowd pleasing romantic movies states that the two leads must end up togetherby film's end .if you're not familiar with this law , then maybe you've seen the trailer for this film which shows that the two leads are together by film's end . now if you're a regular reader of mine , you've heard me say this countless times : you know how drive me crazy is going to end , but is the journey to get to that ending worth it ? no , it definitely is not . melissa joan hart ( from abc's  sabrina , the teenage witch  ) likes a hunky stud on the basketball team . adrien grenier is her grungy neighbor who's just broken up with his activist girlfriend . apparently he wants to make his ex-girlfriend jealous enough to take him back , and she wants someone to take her to the big year end dance ."]
#tes_counts = count_vectorizer.transform(tes)
#predictions = classifier.predict(tes_counts)
#print(predictions)


['jelek' 'bagus']


In [7]:
#opsi1
# membuat pipeline untuk contoh tersebut
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('classifier',  MultinomialNB()) ])
#------------------------------------------------------------

#opsi2
##Memperbaiki akurasinya dengan memperbaiki pipeline. 
##Diterapkan n-gram, 1 sampai 2, atau disebut juga bigram count. Yang berarti satu sampai dua kata yang akan di Vectorize. 
##from sklearn.pipeline import Pipeline

##pipeline = Pipeline([
##    ('count_vectorizer', CountVectorizer(ngram_range=(1, 2))),
##    ('classifier',       MultinomialNB())
##])
#------------------------------------------------------------

#opsi3
###Menerapkan TfidfTransformer.
###from sklearn.feature_extraction.text import TfidfTransformer

###pipeline = Pipeline([
###    ('count_vectorizer',   CountVectorizer(ngram_range=(1,  2))),
###    ('tfidf_transformer',  TfidfTransformer()),
###    ('classifier',         MultinomialNB())
###])
#------------------------------------------------------------

#opsi4
####Menerapkan classifier Bernoulli Naïve Bayes.
####from sklearn.naive_bayes import BernoulliNB

####pipeline = Pipeline([
####    ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
####    ('classifier',         BernoulliNB(binarize=0.0)) ])
#------------------------------------------------------------

pipeline.fit(dataframe['text'], dataframe['class'])
pipeline.predict(examples)

array(['jelek', 'bagus'], dtype='<U5')

In [8]:
# validasi atau model selection KFold
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score

k_fold = KFold(n_splits=6, random_state=None, shuffle=True)
scores = []
confusion = numpy.array([[0, 0], [0, 0]])
for train_indices, test_indices in k_fold.split(dataframe):
    train_text = dataframe.iloc[train_indices]['text']
    train_y = dataframe.iloc[train_indices]['class']

    test_text = dataframe.iloc[test_indices]['text']
    test_y = dataframe.iloc[test_indices]['class']

    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label="jelek")
    scores.append(score)

print('Total review classified:', len(dataframe))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

Total review classified: 1000
Score: 0.8106198824589629
Confusion matrix:
[[395 105]
 [ 87 413]]


In [9]:
#Untuk memprediksi review lainnya, yaitu dengan kode di bawah ini:

tes = ["the law of crowd pleasing romantic movies states that the two leads must end up togetherby film's end .if you're not familiar with this law , then maybe you've seen the trailer for this film which shows that the two leads are together by film's end . now if you're a regular reader of mine , you've heard me say this countless times : you know how drive me crazy is going to end , but is the journey to get to that ending worth it ? no , it definitely is not . melissa joan hart ( from abc's  sabrina , the teenage witch  ) likes a hunky stud on the basketball team . adrien grenier is her grungy neighbor who's just broken up with his activist girlfriend . apparently he wants to make his ex-girlfriend jealous enough to take him back , and she wants someone to take her to the big year end dance ."]
tes_counts = count_vectorizer.transform(tes)
predictions = classifier.predict(tes_counts)
print(predictions)


['jelek']
